### LangChain

LangChain is een open source framework voor het gebruiken van Large Language Models (LLM's). Het idee achter LangChain is dat je verschillende componenten aan elkaar kan koppelen (chaining) zodat je geavanceerde use cases kan ontwikkelen met LLM's. De belangrijkste componenten van deze chains voor deze workshop zijn:
1. Promt templates
2. LLM's.

In [1]:
!pip install langchain
!pip install openai
!pip install python-dotenv

In [2]:
from dotenv import load_dotenv
import os

load_dotenv()

True

In [3]:
from langchain import OpenAI, PromptTemplate, LLMChain

# Basis template voor het beantwoorden van een vraag.
template = """Vraag: {query}

Antwoord:"""

# Creeeren van een promt template op basis van het bovenstaande template.
prompt = PromptTemplate(template=template, input_variables=["query"])

# Het specificeren van een te gebruiken LLM.
gpt = OpenAI(model_name="gpt-3.5-turbo-instruct")

# Een Chain generen op basis van het promt template en de LLM om een vraag te beantwoorden.
llm_chain = LLMChain(prompt=prompt, llm=gpt)

# Een vraag stellen.
question = "Geef voorbeelden van spin +1/2 elementaire deeltjes."
print(llm_chain.run(question))

 Voorbeelden van spin +1/2 elementaire deeltjes zijn onder andere elektronen, positronen, quarks en neutrino's.


### Prompt templates ontwerpen in LangChain

Prompts bevatten vaak een of meer van de volgende componenten:
- *Instructie*. Uitleg aan het model wat het moet doen: hoe het externe informatie moet verwerken (indien van toepassing), wat te doen met de query en hoe het een output moet construeren. 
- *Context*. Exterene bronnen van informatie toevoegen, bijvoorbeeld vanuit een vector database (RAG).
- *Gebruikersinput*. Dit is de input in het systeem van de gebruiker.
- *Outputindicator*. Dit is het begin van de door het model te genereren tekst.

In [4]:
template = """Beantwoord de vraag op basis van de hieronder gegeven context.
Als je de vraag niet kan beantwoorden met de gegeven context, antwoord dan 'Ik weet het antwoord niet'. Ga niet
zelf een antwoord verzinnen, als je de vraag niet kunt beantwoorden met de gegeven context.

Context: DICTU is een van de grootste ICT-dienstverleners binnen de Rijksoverheid. 
Ruim 1.800 gedreven professionals werken in Den Haag, Assen en Zwolle vol enthousiasme en met focus 
op de klant aan voor Nederland belangrijke IT-systemen.
DICTU legt zich toe op het ontwikkelen en beheren van systemen en applicaties en de bijbehorende dienstverlening. 
Dat kunnen bedrijfsvoeringsapplicaties, websites en apps zijn, maar ook digitale werkplekken en applicaties op 
het gebied van subsidies, inspecties en registers. Wij maken daarbij gebruik van moderne technieken en 
ontwikkelingen zoals Cloud, Mobile en Big Data.

Vraag: {query}

Antwoord: """

prompt_template = PromptTemplate(input_variables=["query"], template=template)
print(prompt_template.format(query="Waar heeft DICTU kantoren?"))

Beantwoord de vraag op basis van de hieronder gegeven context.
Als je de vraag niet kan beantwoorden met de gegeven context, antwoord dan 'Ik weet het antwoord niet'. Ga niet
zelf een antwoord verzinnen, als je de vraag niet kunt beantwoorden met de gegeven context.

Context: DICTU is een van de grootste ICT-dienstverleners binnen de Rijksoverheid. 
Ruim 1.800 gedreven professionals werken in Den Haag, Assen en Zwolle vol enthousiasme en met focus 
op de klant aan voor Nederland belangrijke IT-systemen.
DICTU legt zich toe op het ontwikkelen en beheren van systemen en applicaties en de bijbehorende dienstverlening. 
Dat kunnen bedrijfsvoeringsapplicaties, websites en apps zijn, maar ook digitale werkplekken en applicaties op 
het gebied van subsidies, inspecties en registers. Wij maken daarbij gebruik van moderne technieken en 
ontwikkelingen zoals Cloud, Mobile en Big Data.

Vraag: Waar heeft DICTU kantoren?

Antwoord: 


In [5]:
llm_chain = LLMChain(prompt=prompt_template, llm=gpt)

question = "Wat doet DICTU?"
print(f"Vraag: {question}\nAntwoord: {llm_chain.run(question)}\n")

Vraag: Wat doet DICTU?
Antwoord: DICTU legt zich toe op het ontwikkelen en beheren van systemen en applicaties en de bijbehorende dienstverlening.



In [6]:
question = "Wat doet de RVO?"
answer = llm_chain.run(question)
print(f"Vraag: {question}\nAntwoord: {llm_chain.run(question)}\n")

Vraag: Wat doet de RVO?
Antwoord: Ik weet het antwoord niet.



#### Few shot prompt template

In [7]:
from langchain import FewShotPromptTemplate

examples = [
    {"query": "Deze film was walgelijk!",
     "answer": "Negatief"
    },
    {"query": "Deze film was geweldig",
     "answer": "Positief"
    },
    {"query": "Ik vond hem erg goed",
     "answer": "Positief"
    },
    {"query": "Positief",
     "answer": "Positief"
    },
    {"query": "Negatief",
     "answer": "Negatief"
    },
    {"query": "Dit was zo'n slechte film",
     "answer": "Negatief"
    },
    {"query": "Halverwege stopte ik met kijken",
     "answer": "Negatief"
    },
    {"query": "Het was echt slecht",
     "answer": "Negatief"
    }
]

example_template = """
Gebruiker: {query}
AI: {answer}
"""
example_prompt  = PromptTemplate(input_variables=["query", "answer"], template=example_template)

prefix = """Hier zijn voorbeelden van een gesprek tussen een AI en een gebruiker. 
De AI helpt de gebruiker met het classificeren van het sentiment van filmreviews. 
Gegeven een filmreview van de gebruiker antwoord de AI met alleen met een van de woorden 'Positief' of 'Negatief'. 
'Positief' als de review positief is en 'Negatief' als de review negatief is. Geef antwoorden zoals deze AI."""

suffix = """
Gebruiker: {query}
AI: """

few_shot_prompting_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n\n"
)
print(few_shot_prompting_template.format(query="Geweldige film!"))

Hier zijn voorbeelden van een gesprek tussen een AI en een gebruiker. 
De AI helpt de gebruiker met het classificeren van het sentiment van filmreviews. 
Gegeven een filmreview van de gebruiker antwoord de AI met alleen met een van de woorden 'Positief' of 'Negatief'. 
'Positief' als de review positief is en 'Negatief' als de review negatief is. Geef antwoorden zoals deze AI.


Gebruiker: Deze film was walgelijk!
AI: Negatief



Gebruiker: Deze film was geweldig
AI: Positief



Gebruiker: Ik vond hem erg goed
AI: Positief



Gebruiker: Positief
AI: Positief



Gebruiker: Negatief
AI: Negatief



Gebruiker: Dit was zo'n slechte film
AI: Negatief



Gebruiker: Halverwege stopte ik met kijken
AI: Negatief



Gebruiker: Het was echt slecht
AI: Negatief



Gebruiker: Geweldige film!
AI: 


In [8]:
query = "In jaren heb ik niet zo'n goede film gezien"
llm_chain = LLMChain(prompt=few_shot_prompting_template, llm=gpt)
answer = llm_chain.run(query)
print(answer)

Positief


### Vector Databases

Een mooi en simpel voorbeeld van een vectordatabase gebruik makend van LangChain en Pinecone is hier te volgen: https://github.com/pinecone-io/examples/blob/master/learn/generation/langchain/handbook/05-langchain-retrieval-augmentation.ipynb

In [9]:
!pip install datasets
!pip install tiktoken

In [10]:
from datasets import load_dataset

data = load_dataset("wikipedia", "20220301.simple", split='train[:10000]')
turing_data = data[6]

print(turing_data['text'])

/Users/christopherspelt/.pyenv/versions/3.11.0/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Alan Mathison Turing OBE FRS (London, 23 June 1912 – Wilmslow, Cheshire, 7 June 1954) was an English mathematician and computer scientist. He was born in Maida Vale, London.

Early life and family 
Alan Turing was born in Maida Vale, London on 23 June 1912. His father was part of a family of merchants from Scotland. His mother, Ethel Sara, was the daughter of an engineer.

Education 
Turing went to St. Michael's, a school at 20 Charles Road, St Leonards-on-sea, when he was five years old.
"This is only a foretaste of what is to come, and only the shadow of what is going to be.” – Alan Turing.

The Stoney family were once prominent landlords, here in North Tipperary. His mother Ethel Sara Stoney (1881–1976) was daughter of Edward Waller Stoney (Borrisokane, North Tipperary) and Sarah Crawford (Cartron Abbey, Co. Longford); Protestant Anglo-Irish gentry.

Educated in Dublin at Alexandra School and College; on October 1st 1907 she married Julius Mathison Turing, latter son of Reverend Joh

In [11]:
import tiktoken

tokenizer = tiktoken.get_encoding('cl100k_base')

def tiktoken_len(text):
    tokens = tokenizer.encode(text, disallowed_special=())
    return len(tokens)

In [12]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=40,
    chunk_overlap=2,
    length_function=tiktoken_len,
    separators=["\n\n", "\n", " ", ""]
)

chunks = text_splitter.split_text(turing_data['text'])
for i, chunk in enumerate(chunks):
    print(f"chunk {i}: {chunk}")

chunk 0: Alan Mathison Turing OBE FRS (London, 23 June 1912 – Wilmslow, Cheshire, 7 June 1954) was an English mathematician and computer
chunk 1: and computer scientist. He was born in Maida Vale, London.
chunk 2: Early life and family
chunk 3: Alan Turing was born in Maida Vale, London on 23 June 1912. His father was part of a family of merchants from Scotland. His mother, Ethel Sara, was the
chunk 4: was the daughter of an engineer.
chunk 5: Education 
Turing went to St. Michael's, a school at 20 Charles Road, St Leonards-on-sea, when he was five years old.
chunk 6: "This is only a foretaste of what is to come, and only the shadow of what is going to be.” – Alan Turing.
chunk 7: The Stoney family were once prominent landlords, here in North Tipperary. His mother Ethel Sara Stoney (1881–1976) was daughter of Edward Waller Stoney
chunk 8: Stoney (Borrisokane, North Tipperary) and Sarah Crawford (Cartron Abbey, Co. Longford); Protestant Anglo-Irish gentry.
chunk 9: Educated in Dublin at

In [13]:
encoded_chunks = list(map(tokenizer.encode, chunks))

for i, encoded_chunk in enumerate(encoded_chunks):
    print(f"chunk {i}: {encoded_chunk}")

chunk 0: [68724, 4242, 3416, 95530, 507, 11855, 435, 11706, 320, 40672, 11, 220, 1419, 5651, 220, 7529, 17, 1389, 10785, 1026, 10516, 11, 921, 90345, 11, 220, 22, 5651, 220, 6280, 19, 8, 574, 459, 6498, 21651, 1122, 323, 6500]
chunk 1: [438, 6500, 28568, 13, 1283, 574, 9405, 304, 386, 36544, 55069, 11, 7295, 13]
chunk 2: [42298, 2324, 323, 3070]
chunk 3: [68724, 95530, 574, 9405, 304, 386, 36544, 55069, 11, 7295, 389, 220, 1419, 5651, 220, 7529, 17, 13, 5414, 7126, 574, 961, 315, 264, 3070, 315, 42870, 505, 19627, 13, 5414, 6691, 11, 14693, 301, 44020, 11, 574, 279]
chunk 4: [16514, 279, 10003, 315, 459, 24490, 13]
chunk 5: [40234, 720, 51, 1711, 4024, 311, 800, 13, 8096, 596, 11, 264, 2978, 520, 220, 508, 15274, 9728, 11, 800, 21605, 2402, 10539, 7962, 64, 11, 994, 568, 574, 4330, 1667, 2362, 13]
chunk 6: [22244, 374, 1193, 264, 2291, 83, 5642, 315, 1148, 374, 311, 2586, 11, 323, 1193, 279, 12737, 315, 1148, 374, 2133, 311, 387, 2029, 1389, 26349, 95530, 13]
chunk 7: [791, 800, 2596, 